Use commot environemnt

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from tqdm import tqdm

sc.settings.verbosity = 1
sns.set(font_scale=1)
sc.settings.set_figure_params(dpi=150)
sns.set_style("ticks")

from matplotlib import cm
from matplotlib.colors import ListedColormap

cm_color = cm.get_cmap("Reds", 128)
cm_grey = cm.get_cmap("Greys", 128)

Reds = ListedColormap(np.vstack((
    cm_grey(np.linspace(0.2, 0.2, 1)),
    cm_color(np.linspace(0.1, 1, 128)),
)))

In [2]:
datasets = ["A02993E1", 
           "A02989E1", 
           "A02994D6",
           "A02994E6",
           "C01939A4",
           "C01939A5",
           "C01939A6",
           # "C01939B2", # Missing cell.cluseter 
          ]

In [3]:
raw_data_path="/home/felix/data/GEO_submission_facial/stereoseq/"
my_path="/home/felix/projects/facial/felix/data/reprocessed_data/"


In [4]:

for dataset in ['C01939A4','C01939A5', 'C01939A6']:
    adata = sc.read_h5ad(f"{raw_data_path}/{dataset}.cell.cluster.h5ad")
    

    for resolution in [0.5, 1, 1.5]:
        sc.tl.leiden(adata, resolution=resolution, key_added=f"leiden_{resolution}")

    adata.obsm["X_spatial"] = adata.obsm["spatial"].copy()

    del adata.obsm["spatial"]
    del adata.obs["orig.ident"], adata.obs["x"], adata.obs["y"], adata.obs["leiden"]
    del adata.uns["neighbors"], adata.uns["sn"]
    del adata.obsp["connectivities"], adata.obsp["distances"]

    adata_counts = adata.raw.to_adata().copy()
    adata.raw = None
    
    
    adata.write_h5ad(f"{my_path}{dataset}/processed/cells.h5ad")
    adata_counts.write_h5ad(f"{raw_data_path}/{dataset}_raw_cells.h5ad")

    

/home/felix/miniconda3/envs/commot/lib/python3.7/site-packages/anndata/_core/raw.py:146: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  uns=self._adata.uns.copy(),


In [ ]:

for dataset in ["A02989E1", "A02993E1", "A02994D6", "A02994E6"]:

    adata_nonadj = sc.read_h5ad(f"{raw_data_path}/{dataset}.cell.cluster.h5ad")
    adata_adj = sc.read_h5ad(f"{raw_data_path}/{dataset}.adjusted.cell.cluster.h5ad")
    
    for adata in [adata_nonadj, adata_adj]:        
        for resolution in [0.5, 1, 1.5]:
            sc.tl.leiden(adata, resolution=resolution, key_added=f"leiden_{resolution}")
            
            adata.obsm["X_spatial"] = adata.obsm["spatial"].copy()

        del adata.obsm["spatial"]
        del adata.obs["orig.ident"], adata.obs["x"], adata.obs["y"], adata.obs["leiden"]
        for key in list(adata.uns.keys()).copy():
            del adata.uns[key]
        del adata.obsp["connectivities"], adata.obsp["distances"]
        
    adata_nonadj_counts = adata_nonadj.raw.to_adata().copy()
    adata_nonadj.raw = None
    
    adata_adj_counts = adata_adj.raw.to_adata().copy()
    adata_adj.raw = None
    
    adata_adj.write_h5ad(f"{my_path}{dataset}/processed/cells_adjusted.h5ad")
    adata_nonadj.write_h5ad(f"{my_path}{dataset}/processed/cells.h5ad")
    
    adata_adj_counts.write_h5ad(f"{raw_data_path}/{dataset}_raw_cells_adjusted.h5ad")
    adata_nonadj_counts.write_h5ad(f"{raw_data_path}/{dataset}_raw_cells.h5ad")
    